In [1]:
import math
import pandas as pd
import numpy as np
import librosa
import warnings
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scipy.io import wavfile
from collections import OrderedDict
from tqdm import tqdm
import pickle
import json
import glob
import os
from os import path

from PIL import Image

# For producing label map for TF2 Obj Detection
from object_detection.protos import string_int_label_map_pb2
from google.protobuf import text_format

# For saving TFRecords
import tensorflow as tf
from object_detection.utils import dataset_util
import io

# For sharding TFRecords
import contextlib2
from object_detection.dataset_tools import tf_record_creation_util

In [2]:
left_col, right_col = "Begin Time (s)", "End Time (s)"
top_col, bot_col = "High Freq (Hz)", "Low Freq (Hz)"
class_col, class_conf_col = "Species", "Species Confidence"

recording_dir = "../data"
annotation_dir = "../data"
output_dir = "../data/spectrograms-1600Hz-15s-300px-082621"
label_map_name = "label_map.pbtxt"
metadata_name = "dataset_metadata.txt"

# SPECTROGRAM CONSTANTS
# Window size (n_fft) in seconds
WINDOW_SIZE_SEC = 3/20
# Hop Length in seconds
HOP_LEN_SEC = 15/300
# Number of frequency bands (y dimension of spectrogram)
N_MELS = 300
# Maximum frequency considered (highest value in y dimension)
FREQUENCY_MAX = 1600

# CHUNK CONSTANTS
# Length of one chunk in seconds
TRAIN_CHUNK_SIZE_SEC = 45
EVAL_CHUNK_SIZE_SEC = 15
# Minimum % visibility of a call to keep annotation
MIN_BOX_PERCENT = 0.3

# DATASET SETTINGS
dataset_name = "dataset.record"
NUM_TRAIN_SHARDS = 3
NUM_EVAL_SHARDS = 5
NUM_EVAL_FILES = 2

# Constructs the dataset without certain classes
DISALLOWED_CLASSES = ["?", "rf", "sl"]

In [3]:
output_dir = path.abspath(output_dir)
print("Output directory is: '{}'".format(output_dir))

os.makedirs(output_dir)
print("Successfully created directory.")

Output directory is: '/home/jackson/Projects/marine-acoustics-2021/data/spectrograms-1600Hz-15s-300px-082621'
Successfully created directory.


In [4]:
with open(path.join(output_dir, metadata_name), 'w') as metafile:
    json.dump(
        {
            "WINDOW_SIZE_SEC": WINDOW_SIZE_SEC,
            "HOP_LEN_SEC": HOP_LEN_SEC,
            "N_MELS": N_MELS,
            "FREQUENCY_MAX": FREQUENCY_MAX,
            "TRAIN_CHUNK_SIZE_SEC": TRAIN_CHUNK_SIZE_SEC,
            "EVAL_CHUNK_SIZE_SEC": EVAL_CHUNK_SIZE_SEC,
            "EVAL_CHUNK_STEP_SEC": EVAL_CHUNK_SIZE_SEC / 2.0
        },
        metafile
    )

In [5]:
def get_file_id(fname):
    return path.basename(fname)[:22]

# Constructs a list of pairs (wav_fname, annot_fname)
def get_filename_pairs(recording_dir, annotation_dir):
    wav_fnames = glob.glob(path.join(recording_dir, "*.wav"))
    annot_fnames = glob.glob(path.join(annotation_dir, "*.txt"))
    
    res = []
    id_set = set()
    for wfname in wav_fnames:
        id_nums = get_file_id(wfname)
        if id_nums in id_set:
            raise ValueError("Duplicate Wav ID: {}".format(id_nums))
        id_set.add(id_nums)
        
        annots = [a for a in annot_fnames if path.basename(a).startswith(id_nums)]
        if len(annots) > 1:
            raise ValueError("More than one annotation for recording: {}".format(wfname))
        if len(annots) < 1:
            print("No annotation for recording: {}".format(wfname))
            continue
        
        res.append((wfname, annots[0]))
    
    return res

In [6]:
dataset = get_filename_pairs(recording_dir, annotation_dir)
dataset

No annotation for recording: ../data/671658014.181001003530_norm_8k-resample.wav
No annotation for recording: ../data/671658014.180930213531_norm_8k-resample.wav
No annotation for recording: ../data/671658014.181001033528_norm_8k-resample.wav


[('../data/671658014.180930063538_norm_8k-resample.wav',
  '../data/671658014.180930063538-MS.txt'),
 ('../data/671658014.180930003542_norm_8k-resample.wav',
  '../data/671658014.180930003542-AW.txt'),
 ('../data/671658014.180930033540_norm_8k-resample.wav',
  '../data/671658014.180930033540-AW.txt'),
 ('../data/671658014.180929003601_norm_8k-resample.wav',
  '../data/671658014.180929003601-MS.txt'),
 ('../data/671658014.180929183547_norm_8k-resample.wav',
  '../data/671658014.180929183547-MS.txt'),
 ('../data/671658014.180930123535_norm_8k-resample.wav',
  '../data/671658014.180930123535-MS.txt'),
 ('../data/671658014.180928183606_norm_8k-resample.wav',
  '../data/671658014.180928183606-AW.txt'),
 ('../data/671658014.180929123551_norm_8k-resample.wav',
  '../data/671658014.180929123551-AW.txt'),
 ('../data/671658014.180929213545_norm_8k-resample.wav',
  '../data/671658014.180929213545-AW.txt'),
 ('../data/671658014.180928213604_norm_8k-resample.wav',
  '../data/671658014.180928213604-

In [7]:
#eval_indices = np.random.choice(len(dataset), NUM_EVAL_FILES, replace=False)
#eval_indices = np.array([0, 2, 3, 4, 5, 6])

#train_dataset = [e for i,e in enumerate(dataset) if i not in eval_indices]
#eval_dataset = [e for i,e in enumerate(dataset) if i in eval_indices]

#print("{} Train files and {} Eval files". format(len(train_dataset), len(eval_dataset)))

In [8]:
# Manually Set

train_dataset = [
    ('../data/671658014.180929033558_norm_8k-resample.wav', '../data/671658014.180929033558-JW.txt'),
    ('../data/671658014.180929003601_norm_8k-resample.wav', '../data/671658014.180929003601-MS.txt'),
    ('../data/671658014.180928153609_norm_8k-resample.wav', '../data/671658014.180928153609-JW.txt'),
    ('../data/671658014.180929123551_norm_8k-resample.wav', '../data/671658014.180929123551-AW.txt'),
    ('../data/671658014.180930003542_norm_8k-resample.wav', '../data/671658014.180930003542-AW.txt'),
    ('../data/671658014.180930063538_norm_8k-resample.wav', '../data/671658014.180930063538-MS.txt'),
    ('../data/671658014.180930033540_norm_8k-resample.wav', '../data/671658014.180930033540-AW.txt'),
    ('../data/671658014.180930123535_norm_8k-resample.wav', '../data/671658014.180930123535-MS.txt'),
    ('../data/671658014.180929213545_norm_8k-resample.wav', '../data/671658014.180929213545-AW.txt'),
    ('../data/671658014.180930183532_norm_8k-resample.wav', '../data/671658014.180930183532-MS.txt'),
    ('../data/671658014.180929153549_norm_8k-resample.wav', '../data/671658014.180929153549-MS.txt')
]

eval_dataset = [
    ('../data/671658014.180928183606_norm_8k-resample.wav', '../data/671658014.180928183606-AW.txt'),
    ('../data/671658014.180928213604_norm_8k-resample.wav', '../data/671658014.180928213604-MS.txt'),
    ('../data/671658014.180929063556_norm_8k-resample.wav', '../data/671658014.180929063556-MS.txt'),
    ('../data/671658014.180929093553_norm_8k-resample.wav', '../data/671658014.180929093553-MS.txt'),
    ('../data/671658014.180929183547_norm_8k-resample.wav', '../data/671658014.180929183547-MS.txt'),
    ('../data/671658014.180930153534_norm_8k-resample.wav', '../data/671658014.180930153534-AW.txt')
]

In [9]:
# TODO: measure speed of different fns for opening wav files
def read_wavfile(wav_name, normalize=True, verbose=False):
    if verbose:
        print("Reading {}".format(wav_name))
    sr, data = wavfile.read(wav_name)
    if verbose:
        print("{} samples at {} samples/sec --> {} seconds".format(data.shape[0], sr, data.shape[0]/sr))

    if normalize:
        data = data.astype(float)
        data = data - data.min()
        data = data / data.max()
        data = data - 0.5
    
    return sr, data


def read_annotations(fname, verbose=False):
    annotations = pd.read_csv(fname, sep="\t")
    if verbose:
        print("Read {} annotations from {}".format(len(annotations), fname))
        print("Columns:", ",".join([" {} ({})".format(c, type(c)) for c in annotations.columns]))
    return annotations

In [10]:
def get_all_classes(annotation_fnames, verbose=False):
    """
    Returns a list of all classes seen in the annotation files sorted
    alphabetically.
    """
    classes = set()
    for annot_fname in annotation_fnames:
        classes.update(list(read_annotations(annot_fname)[class_col].unique()))
    classes = sorted([s for s in list(classes)])
    if verbose:
        print("Classes: ", classes)
    return classes


# Generates the necessary prototext file for the class mapping.
# Classes are assigned to the integer 1 greater than their index.
# The resulting file is saved to output_path.
def create_label_map(classes, output_path):
    label_map = string_int_label_map_pb2.StringIntLabelMap()
    for i, cls in enumerate(classes):
        new_item = label_map.item.add() # StringIntLabelMapItem
        new_item.name = cls          # String name. The most common practice is to set this to a MID or synsets id.
        new_item.id = 1+i            # Integer id starting from 1
        new_item.display_name = cls  # Human readable text label
    with open(output_path, "w") as f:
        f.write(text_format.MessageToString(label_map))
        

classes = get_all_classes([a for _,a in dataset], verbose=True)
classes = [c for c in classes if c not in DISALLOWED_CLASSES]
create_label_map(classes, os.path.join(output_dir, label_map_name))


# The class and reverse class maps are used to encode classes later.
class_map = {}
rev_class_map = {}
for i in range(len(classes)):
    class_map[i+1] = classes[i]
    rev_class_map[classes[i]] = i+1

Classes:  ['?', 'hb', 'rf', 'sl']


In [14]:
def get_area(annotation):
    return ((annotation[right_col] - annotation[left_col])
            * (annotation[top_col] - annotation[bot_col]))


# Per-channel energy normalization
def PCEN(spec, M_return_timestep, init_val=None, epsilon=1e-6, s=0.001, alpha=0.80, delta=2.0, r=0.5):
    output = np.zeros_like(spec)
    if M_return_timestep < 0 or M_return_timestep > spec.shape[1]-1:
        print("Warning! M return timestep is outside bounds. Not returning any M.")
    if init_val is None:
        M = np.zeros(shape=(output.shape[0]))
    else:
        M = np.array(init_val)
    assert M.shape[0] == output.shape[0]
    out_M = None
    for t in range(output.shape[1]):
        M = (1 - s) * M + s * spec[:,t]
        output[:,t] = ((spec[:,t] / ((M + epsilon) ** alpha)) ** r) - (delta ** r)
        if t == M_return_timestep:
            out_M = M
    return output, out_M


# Returns the min and max db observed in all wav files
def get_minmax_bounds(wav_filenames, chunk_size=TRAIN_CHUNK_SIZE_SEC):
    min_val, max_val = None, None
    for wfname in wav_filenames:
        sr, data = read_wavfile(wfname, normalize=True)
        n_fft = int(WINDOW_SIZE_SEC * sr)
        hop_len = int(HOP_LEN_SEC * sr)
        chunk_size = int(chunk_size * sr)
        step = chunk_size - (hop_len * (N_MELS-2) + n_fft)
        M_init = None
        for start_i in range(0, len(data), step):
            mel_spec = librosa.feature.melspectrogram(y=data[start_i:min(len(data),start_i+chunk_size)],
                                                      sr=sr,
                                                      n_fft=n_fft,
                                                      hop_length=hop_len,
                                                      n_mels=N_MELS,
                                                      fmax=FREQUENCY_MAX,
                                                      center=False)
            #mel_spec, M_init = PCEN(mel_spec, step // hop_len, init_val=M_init)
            mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
            temp_min = mel_spec.min()
            temp_max = mel_spec.max()
            if min_val is None or temp_min < min_val:
                min_val = temp_min
            if max_val is None or temp_max > max_val:
                max_val = temp_max
    return min_val, max_val


# Creates the tf.train.Example from the example_dict saved for each spectrogram chunk
# The image bytes are read in from disk at this point.
def create_tf_example(example_dict):
    with open(example_dict["filepath"], "rb") as f:
        encoded_image_data = f.read()
    filename = path.basename(example_dict["filepath"]).encode()
    classes_text = [s.encode() for s in example_dict["classes_text"]]
    tf_example = tf.train.Example(features=tf.train.Features(feature={
              'image/height': dataset_util.int64_feature(example_dict["height"]),
              'image/width': dataset_util.int64_feature(example_dict["width"]),
              'image/filename': dataset_util.bytes_feature(filename),
              'image/source_id': dataset_util.bytes_feature(filename),
              'image/encoded': dataset_util.bytes_feature(encoded_image_data),
              'image/format': dataset_util.bytes_feature(b'png'),
              'image/object/bbox/xmin': dataset_util.float_list_feature(example_dict["xmins"]),
              'image/object/bbox/xmax': dataset_util.float_list_feature(example_dict["xmaxs"]),
              'image/object/bbox/ymin': dataset_util.float_list_feature(example_dict["ymins"]),
              'image/object/bbox/ymax': dataset_util.float_list_feature(example_dict["ymaxs"]),
              'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
              'image/object/class/label': dataset_util.int64_list_feature(example_dict["classes"]),
          }))
    return tf_example


def process_file(wav_filename, annot_filename, min_bound, max_bound, chunk_size, chunk_layout="dense",
                 drop_last_chunk=False, verbose=False):
    sr, data = read_wavfile(wav_filename, normalize=True, verbose=verbose)
    annotations = read_annotations(annot_filename, verbose=verbose)
    file_id = get_file_id(wav_filename)
    
    n_fft = int(WINDOW_SIZE_SEC * sr)
    hop_len = int(HOP_LEN_SEC * sr)
    chunk_size = int(chunk_size * sr)
    
    if chunk_layout == "dense":
        step = chunk_size - (hop_len * (N_MELS-2) + n_fft)
    elif chunk_layout == "sparse":
        step = chunk_size // 2
    
    # Start Indices of each chunk
    start_vals = [s for s in range(0, len(data), step)]
    
    # If last cut point creates a tiny chunk, remove it
    if len(data) - start_vals[-1] < int(chunk_size / 2):
        start_vals = start_vals[:-1]
        
    def extract_chunk(start_i, end_i, spec_name, annot_name, use_pcen=True, M_init=None):
        mel_spec = librosa.feature.melspectrogram(y=data[start_i:end_i],
                                                  sr=sr,
                                                  n_fft=n_fft,
                                                  hop_length=hop_len,
                                                  n_mels=N_MELS,
                                                  fmax=FREQUENCY_MAX,
                                                  center=False)
        #mel_spec, next_M_init = PCEN(mel_spec, step // hop_len, init_val=M_init)
        next_M_init = None
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        mel_spec = np.clip((mel_spec - min_bound) / (max_bound - min_bound) * 255, a_min=0, a_max=255)
        mel_spec = mel_spec.astype(np.uint8)
        spec_height, spec_width = mel_spec.shape
        
        # Get annotations to those inside chunk
        start_s, end_s = start_i/sr, end_i/sr
        freq_axis_low, freq_axis_high = librosa.hz_to_mel(0.0), librosa.hz_to_mel(FREQUENCY_MAX)
        chunk_annotations = annotations.loc[~((annotations[left_col] > end_s)
                                              | (annotations[right_col] < start_s))].copy()
        # Rescale axes to 0.0-1.0 based on location inside chunk
        chunk_annotations.loc[:,[left_col,right_col]] = ((chunk_annotations[[left_col,right_col]]
                                                         - start_s) / (end_s - start_s))
        chunk_annotations.loc[:,[bot_col,top_col]] = (1.0 - ((librosa.hz_to_mel(chunk_annotations[[bot_col,top_col]])
                                                      - freq_axis_low) / (freq_axis_high - freq_axis_low)))
        chunk_annotations = chunk_annotations.loc[chunk_annotations[class_col].isin(classes)]
        trimmed_annots = chunk_annotations.copy()
        trimmed_annots[left_col] = trimmed_annots[left_col].clip(lower=0, upper=1.0)
        trimmed_annots[right_col] = trimmed_annots[right_col].clip(lower=0, upper=1.0)
        trimmed_annots[bot_col] = trimmed_annots[bot_col].clip(lower=0, upper=1.0)
        trimmed_annots[top_col] = trimmed_annots[top_col].clip(lower=0, upper=1.0)
        overlaps = []
        for i in trimmed_annots.index:
            intersection = trimmed_annots.loc[i]
            original = chunk_annotations.loc[i]
            original_area = get_area(original)
            overlaps.append((get_area(intersection)*spec_height*spec_width) / original_area)
        chunk_annotations = trimmed_annots.loc[np.array(overlaps) > MIN_BOX_PERCENT]
        
        if verbose:
            print("Found {} annotations in chunk".format(len(chunk_annotations)))
        
        # Save Chunk as PNG image (lossless compression)
        im = Image.fromarray(mel_spec[::-1, :])
        im = im.convert("L")
        image_filepath = path.join(output_dir, spec_name)
        im.save(image_filepath)
        
        if verbose:
            print("Saved spectrogram to '{}'".format(spec_name))
        
        example_dict = {
            "filepath": image_filepath,
            "height": spec_height,
            "width": spec_width,
            "xmins": trimmed_annots[left_col].tolist(),
            "xmaxs": trimmed_annots[right_col].tolist(),
            "ymins": trimmed_annots[top_col].tolist(),
            "ymaxs": trimmed_annots[bot_col].tolist(),
            "classes_text": trimmed_annots[class_col].tolist(),
            "classes": trimmed_annots[class_col].map(rev_class_map).tolist()
        }
        return example_dict, next_M_init
    
    
    # Actually iterate through the file and extract chunks
    examples = []
    M_init = None
    for ind, start_i in enumerate(start_vals[:-1]):
        spec_name = "{}-{}.png".format(file_id, ind)
        annot_name = "{}-{}-labels.txt".format(file_id, ind)
        ex, M_init = extract_chunk(start_i, start_i+chunk_size, spec_name, annot_name, M_init=M_init)
        examples.append(ex)
    if not drop_last_chunk:
        spec_name = "{}-{}.png".format(file_id, len(start_vals)-1)
        annot_name = "{}-{}-labels.txt".format(file_id, len(start_vals)-1)
        ex, _ = extract_chunk(start_vals[-1], len(data), spec_name, annot_name, M_init=M_init)
        examples.append(ex)
    return examples

In [12]:
# Computing scaling parameters based on TRAIN set only.
min_bound, max_bound = get_minmax_bounds([p[0] for p in train_dataset])
print(min_bound, max_bound)

-80.0 0.0


In [15]:
splits = [
    ("train", train_dataset, NUM_TRAIN_SHARDS, TRAIN_CHUNK_SIZE_SEC, "dense", False),
    ("eval", eval_dataset, NUM_EVAL_SHARDS, EVAL_CHUNK_SIZE_SEC, "sparse", True)
]

for splitname, dataset, num_shards, chunk_size, chunk_layout, drop_last_chunk in splits:
    examples = []
    for wav_filename, annot_filename in dataset:
        examples.extend(process_file(wav_filename, annot_filename, min_bound, max_bound, chunk_size,
                                     chunk_layout, drop_last_chunk, verbose=False))

    with contextlib2.ExitStack() as tf_record_close_stack:
        output_filebase = os.path.join(output_dir, "{}_{}".format(splitname, dataset_name))
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
            tf_record_close_stack, output_filebase, num_shards)
        for index, example in enumerate(examples):
            tf_example = create_tf_example(example)
            output_shard_index = index % num_shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())